<a href="https://colab.research.google.com/github/nghoanglong/DS102-M11-CNCL/blob/main/Marketing_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libaries và Packages cần thiết

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json

from datetime import date
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
%matplotlib inline

In [ ]:
# Kết nối tới Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DOWNLOAD_URL = 'https://raw.githubusercontent.com/nghoanglong/DS102-M11-CNCL/main/data/marketing_data.csv'
SAVING_PATH = '/content/drive/MyDrive/Colab Notebooks/HMTK/dataset'

# Tiền xử lý dữ liệu - Hoàng Long
**Lưu ý:** Sau khi đã chạy các bước tiền xử lý dữ liệu này rồi, data được tiền xử lý sẽ được lưu tại một **SAVING_PATH** cố định, ta chỉ cần load data này lên và xây dựng model, không cần chạy lại các bước tiền xử lý này nữa

## Down and Save Dataset
**Lưu ý:** Nếu chưa down và save dataset, chạy các scripts dưới đây. Nếu đã từng chạy các scripts dưới đây, vui lòng bỏ qua và làm các bước tiếp theo

In [ ]:
def fetch_data(download_url=DOWNLOAD_URL, saving_path=SAVING_PATH):
    os.makedirs(saving_path, exist_ok=True)
    tgz_path = os.path.join(saving_path, 'marketing_analytics.csv')
    urllib.request.urlretrieve(download_url, tgz_path)

In [ ]:
fetch_data(DOWNLOAD_URL, SAVING_PATH)

## Load data
Đưa dữ liệu vào DataFrame

In [ ]:
DATASET_PATH = os.path.join(SAVING_PATH, 'marketing_analytics.csv')

In [ ]:
df = pd.read_csv(DATASET_PATH)

In [ ]:
# kiểm tra dữ liệu với 5 dòng đầu
df.head(5)

In [ ]:
# Xem thông tin về dữ liệu
df.info()

## Rename Attributes and Cast Type
Ở bước này, ta nhận thấy có một số vấn đề sau với dữ liệu:

*   Tên của các cột chưa được chuẩn hóa (Sai và gây khó hiểu)
*   Kiểu dữ liệu của các cột chưa được tốt

Ta tiến hành đổi tên các cột, và chuyển đổi kiểu dữ liệu như sau



In [ ]:
# Rename columns
df.rename(columns={'ID':'User_ID'}, inplace=True)
df.rename(columns={' Income ':'Income'}, inplace=True)
df.rename(columns={'Dt_Customer': 'Date_Enroll'}, inplace = True)
df.rename(columns={'MntMeatProducts': 'MntMeats'}, inplace = True)
df.rename(columns={'MntFishProducts': 'MntFishs'}, inplace = True)
df.rename(columns={'MntSweetProducts': 'MntSweets'}, inplace = True)
df.rename(columns={'MntGoldProds': 'MntGolds'}, inplace = True)

In [ ]:
# Cast type to numertical
df['Income'] = df['Income'].str.replace('$', '')
df['Income'] = df['Income'].str.replace(',', '').astype('float')

# Cast type to datetime
df['Date_Enroll'] = pd.to_datetime(df['Date_Enroll'])

In [ ]:
# Kết quả
df.info()

## Explory Data Analysis
Phân tích dữ liệu, đưa ra góc nhìn tổng quan về dữ liệu

In [ ]:
# Tổng quan thông số dữ liệu
df.describe()

### Xử lý Outliers của các cột dạng Numeric

In [ ]:
# Lấy ra những cột dạng numeric trong data
numeric = df.select_dtypes(exclude='object')

In [ ]:
cols_to_check = [col for col in numeric.columns[0:18] if col not in ['User_ID', 'Date_Enroll']]
cols_to_check

In [ ]:
# visualize outliers
fig, ax = plt.subplots(4, 4, figsize = (20, 10))
ax = ax.flatten()
for i, c in enumerate(cols_to_check):
    sns.boxplot(x = df[c], ax = ax[i])
fig.tight_layout()

Ta thấy đa phần dữ liệu đều có những outliners nhưng dữ liệu tại cột Year_Birth mang những outliners không đúng logic. Tính tại năm hiện tại (2021) thì những người sinh năm <= 1900 đã vượt ngoài 120 tuổi. Ta tiến hành drop hết những người có độ tuổi là dữ liệu lỗi

In [ ]:
# drop những người có độ tuổi trên 100 ~ year_birth <= 1900
class HandleOutliners:
    def __init__(self, outliners = True): # no *args or **kargs
        self.outliners = outliners
    def fit(self, X, y=None):
        return self # nothing else to do
    def transform(self, X):
        dropped = X.copy()
        if 'Year_Birth' in X.columns:
          dropped = X[numeric["Year_Birth"]>1900].reset_index(drop=True)
        return dropped

### Xử lý dữ liệu Null

In [ ]:
# Plot các cột để kiểm tra Null
missing = df.isna().sum().reset_index()
missing.columns = ['features', 'total_missing']
missing.index = missing['features']
del missing['features']
missing['total_missing'].plot(kind = 'bar', figsize=(14, 4))

In [ ]:
# Xử lý dữ liệu Null
class HandlingNullData:
    def __init__(self, handle=True): # no *args or **kargs
        self.handle = handle
    def fit(self, X, y=None):
        return self # nothing else to do
    def transform(self, X):
        cp_df = X.copy()
        if 'Income' in X.columns:
          median = cp_df['Income'].median()
          cp_df['Income'].fillna(median, inplace=True)
        return cp_df

Ta thấy cột Income mang giá trị Null, ta tiến hành fill các giá trị mean vào những chỗ có mang giá trị Null này

### Build Pipeline Preprocessing
Ta cần build một pipeline có khả năng xử lý outliners, fill các mean vào những sample mang dữ liệu Null, áp dụng Standard Scaler

In [ ]:
# drop cột Date_Enroll
df = df.drop('Date_Enroll', axis=1)

In [ ]:
# Combine những thuộc tính cần thiết 
df['Total_Spent'] = df[[col for col in df.columns if 'Mnt' in col]].sum(axis=1)
df['TotalPurchases'] = df[[col for col in df.columns if 'Purchases' in col]].sum(axis=1)
df['TotalCampaignsAcc'] = df[[col for col in df.columns if 'Cmp' in col] + ['Response']].sum(axis=1)

In [ ]:
# Xem mối tương quan dữ liệu
plt.figure(figsize=(24,10))
sns.heatmap(df.corr().sort_values(by='Response', ascending=False),annot=True)

Bài toán cần dự đoán nhãn của cột Response, dựa vào biểu đồ tương quan, mức threshold ta cần giữ sẽ >= 0.14

In [ ]:
cols_drop = ['Response', 'Income', 'MntFruits', 'MntSweets', 'MntFishs', 'NumStorePurchases', 
             'Year_Birth', 'Complain', 'NumWebVisitsMonth', 'User_ID', 'Kidhome', 'Teenhome', 'Recency']
selected_cols_data = df.drop(cols_drop,axis=1)
y_label = df['Response']

In [ ]:
selected_cols_data.info()

In [ ]:
# Lấy ra list những categorical columns
cate_cols = list(selected_cols_data.select_dtypes(include='object'))
cate_cols

['Education', 'Marital_Status', 'Country']

In [ ]:
# Lấy ra list những columns ở dạng number
num_cols = list(col for col in selected_cols_data.select_dtypes(exclude='object'))
num_cols

In [ ]:
clean_data = Pipeline([
    ('outliners', HandleOutliners()),
    ('null', HandlingNullData()),
])

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

transform_cols = ColumnTransformer([
    ("num", num_pipeline, num_cols),
    ("cat", OneHotEncoder(handle_unknown='ignore'), cate_cols),
], remainder='passthrough')

full_pipeline = Pipeline([
    ('clean_data', clean_data),
    ('transform_cols', transform_cols),
])

In [ ]:
X = full_pipeline.fit_transform(selected_cols_data)

In [ ]:
print(f'input shape = {X.shape}')
print(f'label shape = {y_label.shape}')

input shape = (2240, 35)
label shape = (2240,)


### Lưu dữ liệu đã được xử lý

In [ ]:
# Lưu input đầu vào
np.save(os.path.join(SAVING_PATH, 'input_data.npy'), X)

In [ ]:
# Lưu label
np.save(os.path.join(SAVING_PATH, 'label.npy'), y_label)

# Xây dựng model - Quỳnh Hoa, Hoàng Long

In [ ]:
input = np.load(os.path.join(SAVING_PATH, 'input_data.npy'))

In [ ]:
input.shape

In [ ]:
y = np.load(os.path.join(SAVING_PATH, 'label.npy'))

In [ ]:
y.shape

## Logistic regression

### Chọn tham số tối ưu

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
parameters = [{'C':[0.001, 0.01, 0.1, 1.0, 10.0], 
               'solver':['sag', 'liblinear', 'lbfgs']}]

grid_search = GridSearchCV(estimator = logreg,  
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 3,
                           verbose=0)

In [ ]:
grid_search.fit(input, y)

In [ ]:
grid_search.cv_results_

### Train and Evaluate

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

skfolds = StratifiedKFold(n_splits=3, random_state=1712, shuffle=True)
logreg = LogisticRegression(C=0.01, solver='liblinear', random_state=1711)
accuracy_lr = []; precision_lr = []; recall_lr = []; f1_lr = []

for train_index, test_index in skfolds.split(input, y):
    X_train_folds = input[train_index]
    y_train_folds = y[train_index]
    X_test_fold = input[test_index]
    y_test_fold = y[test_index]
    logreg.fit(X_train_folds, y_train_folds)
    y_pred = logreg.predict(X_test_fold)
    accuracy_lr.append(accuracy_score(y_test_fold, y_pred))
    precision_lr.append(precision_score(y_test_fold, y_pred))
    recall_lr.append(recall_score(y_test_fold, y_pred))
    f1_lr.append(f1_score(y_test_fold, y_pred))

In [ ]:
print(f'accuracy = {np.mean(accuracy_lr)}')
print(f'precision = {np.mean(precision_lr)}')
print(f'recall = {np.mean(recall_lr)}')
print(f'f1 = {np.mean(f1_lr)}')

## Decision tree

### Chọn tham số tối ưu

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

clf_gini = DecisionTreeClassifier()
parameters = [{'criterion':['gini', 'entropy'], 'max_depth':[1, 2, 3, 4, 5, 6]}]

grid_search = GridSearchCV(estimator = clf_gini,  
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 3,
                           verbose=0)

In [ ]:
grid_search.fit(input, y)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid=[{'criterion': ['gini', 'entropy'],
                          'max_depth': [1, 2, 3, 4, 5, 6]}],
             scoring='accuracy')

In [ ]:
grid_search.cv_results_

### Train and evaluate

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

skfolds = StratifiedKFold(n_splits=3, random_state=1712, shuffle=True)
clf_gini = DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=1711)
accuracy_dt = []; precision_dt = []; recall_dt = []; f1_dt = []

for train_index, test_index in skfolds.split(input, y):
    X_train_folds = input[train_index]
    y_train_folds = y[train_index]
    X_test_fold = input[test_index]
    y_test_fold = y[test_index]
    clf_gini.fit(X_train_folds, y_train_folds)
    y_pred = clf_gini.predict(X_test_fold)
    accuracy_dt.append(accuracy_score(y_test_fold, y_pred))
    precision_dt.append(precision_score(y_test_fold, y_pred))
    recall_dt.append(recall_score(y_test_fold, y_pred))
    f1_dt.append(f1_score(y_test_fold, y_pred))

In [ ]:
print(f'accuracy = {np.mean(accuracy_dt)}')
print(f'precision = {np.mean(precision_dt)}')
print(f'recall = {np.mean(recall_dt)}')
print(f'f1 = {np.mean(f1_dt)}')

## XGBoost

### Chọn tham số tối ưu

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

model_xgb= XGBClassifier()
parameters = {'n_estimators':[10, 20, 30, 40, 50, 100, 200],'max_depth':[3, 4, 5, 6],'learning_rate':[0.001, 0.01, 0.1, 0.01]}
model_xgb_grid = GridSearchCV(model_xgb, 
                              parameters,
                              cv=3,
                              verbose=0)

In [ ]:
model_xgb_grid.fit(input, y)

GridSearchCV(cv=3, estimator=XGBClassifier(),
             param_grid={'learning_rate': [0.001, 0.01, 0.1, 0.01],
                         'max_depth': [3, 4, 5, 6],
                         'n_estimators': [10, 20, 30, 40, 50, 100, 200]})

In [ ]:
model_xgb_grid.cv_results_

In [ ]:
model_xgb_grid.best_params_

### Train and evaluate

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

skfolds = StratifiedKFold(n_splits=3, random_state=1712, shuffle=True)
xgb_classifier = XGBClassifier(n_estimators=200, max_depth=3, learning_rate=0.1, random_state=1711)
accuracy_xgb = []; precision_xgb = []; recall_xgb = []; f1_xgb = []

for train_index, test_index in skfolds.split(input, y):
    X_train_folds = input[train_index]
    y_train_folds = y[train_index]
    X_test_fold = input[test_index]
    y_test_fold = y[test_index]
    xgb_classifier.fit(X_train_folds, y_train_folds)
    y_pred = xgb_classifier.predict(X_test_fold)
    accuracy_xgb.append(accuracy_score(y_test_fold, y_pred))
    precision_xgb.append(precision_score(y_test_fold, y_pred))
    recall_xgb.append(recall_score(y_test_fold, y_pred))
    f1_xgb.append(f1_score(y_test_fold, y_pred))

In [ ]:
print(f'accuracy = {np.mean(accuracy_xgb)}')
print(f'precision = {np.mean(precision_xgb)}')
print(f'recall = {np.mean(recall_xgb)}')
print(f'f1 = {np.mean(f1_xgb)}')